Trying some easy-enough variable PSF so I can write some tests!

In [1]:
%matplotlib notebook
from astropy.io import fits
from astropy.table import Table
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
class PsfEx(object):
    def __init__(self, properties, prop_groups, group_degrees, offsets, scales, coefs):
        assert len(properties) == len(prop_groups)
        assert len(offsets) == len(properties)
        assert len(scales) == len(properties)
        
        self._n_axis = len(properties)
        self._names = properties
        self._groups = prop_groups
        self._n_groups = len(group_degrees)
        self._offsets = offsets
        self._scales = scales
        self._coefs = coefs
        self._n_coefs = len(coefs)
        self._degrees = group_degrees

        self._width = coefs[0].shape[0]
        self._height = coefs[0].shape[1]
        
        if len(coefs) > 1:
            self._powers = self._calculate_powers()
            
    def _calculate_powers(self):
        exponents = np.zeros(self._n_axis, dtype=np.int)
        powers = np.zeros((self._n_coefs, self._n_axis), dtype=np.int)
        group_exponents = np.zeros(self._n_groups, dtype=np.int)
        group_exponents[:self._n_groups] = self._degrees
        
        # Constant
        powers[0,:] = 0
        if self._n_axis:
            group_exponents[self._groups[0]] -= 1
        
        # Polynom
        exponents[0] = 1
        pi = 1
        for t in reversed(range(1, self._n_coefs)):
            powers[pi,:] = exponents[:]
            pi += 1
            
            ei = 0
            for group in self._groups:
                if group_exponents[group]:
                    group_exponents[group] -= 1
                    exponents[ei] += 1
                    break
                else:
                    group_exponents[group] = exponents[ei]
                    exponents[ei] = 0
                    ei += 1
        
        return powers
    
    def get_size(self):
        return self._width, self._height
    
    def normalize_properties(self, *args):
        out = list()
        for i, a in enumerate(args):
            out.append((a - self._offsets[i]) / self._scales[i])
        return out
    
    def get_repr(self, i):
        if i == 0:
            return 'Constant'
        
        components = []
        for v, p in zip(self._names, self._powers[i,:]):
            if p != 0:
                components.append(f'{v}$^{p}$')
            
        return ' + '.join(components)
    
    def get_psf_scaled(self, *args):
        result = np.array(self._coefs[0], copy=True)
        for i in range(1, self._n_coefs):
            exp = self._powers[i,:]
            coefs = self._coefs[i]
            acc = 1
            for v, e in zip(args, exp):
                acc *= v ** e
            result += coefs * acc
        return result
    
    def get_psf(self, *args):
        properties = self.normalize_properties(*args)
        return self.get_psf_scaled(*properties)

In [5]:
constant = np.array([
 [ 0.02333081, 0.03529203, 0.04109584],
 [ 0.03606414, 0.05638209, 0.066116  ],
 [ 0.04230112, 0.06618657, 0.07744237]
])
x = np.array([
 [ 0.00293485, 0.00372489, 0.00018322],
 [ 0.00434255, 0.00756315, 0.00347642],
 [ 0.00227406, 0.00663252, 0.00340644]
])
x2 = np.array([
 [ 0.00116871, 0.00183678,  0.00208775],
 [ 0.00222232, 0.00443414,  0.00414549],
 [ 0.00158169, 0.00286078, -0.00041783]
])
psfex = PsfEx(
    ['x', 'y'], [0, 0], [1],
    offsets=[2040.414154053, 1962.621307373],
    scales=[2924.028137207, 3297.873596191],
    coefs=[constant, x, x2]
)

In [8]:
psfex.get_psf(362.53350830078125, 61.493389129638672)

array([[ 0.02097299,  0.03209574,  0.03978718],
       [ 0.03229117,  0.04948601,  0.06173139],
       [ 0.04008441,  0.06073151,  0.07572854]])